In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 🎯**Capstone Project Submission: Disaster Crisis Logistics Agent**

### Self-Introduction & Project Overview 
**Hello, my name is Isac Jenishraj, and I am a participant in the Google X Kaggle 5-Day AI Agents Intensive Course.**


**I have selected the Agents for Good track for your Capstone.This project is a Disaster Crisis Logistics Agent.This project uses the Function Calling method taught in the Google x Kaggle "AI Agents" course (Day 3), which is the specific architecture they are looking for. Instead of just "chatting," this Disaster Crisis Logistics Agent effectively uses tools to query a supply database and coordinate resource allocation for the Agents for Good track.**


## 📢 1. The Pitch

### **The Problem**
In the immediate aftermath of natural disasters (earthquakes, floods, hurricanes), **information overload** paralyzes response teams. Critical data regarding medical supplies, food packs, and rescue equipment is often trapped in spreadsheets or siloed databases. Field responders waste minutes—sometimes hours—manually querying inventory managers to find resources, while every second counts.

### **The Solution**
**CrisisLogistics Agent** is an autonomous AI coordinator that bridges the gap between chaos and data. Unlike a standard chatbot, this Agent is equipped with **Function Calling capabilities** that allow it to:
1.  **Interpret** urgent, unstructured natural language requests (e.g., *"We need water in Zone A, now!"*).
2.  **Query** a live inventory database autonomously using defined tools.
3.  **Synthesize** precise answers regarding availability and location.

### **Core Value**
* **Reduced Response Time:** Eliminates manual database lookups.
* **Accessibility:** Allows non-technical field workers to interact with complex inventory systems using plain English.
* **Scalability:** Can handle thousands of queries simultaneously, unlike human dispatchers.

## Agent Architecture

* **Brain:** `gemini-1.5-flash` (Optimized for low latency and tool usage).
* **Tools:** Python functions bound to the model, simulating SQL database queries.
* **Architecture:** The model detects when to stop "chatting" and start "doing" by triggering the `check_inventory` function.

## LET'S BEGIN 

**Step1:** Install & Import Libraries and using official Google GenAI SDK.

In [2]:
!pip install -q -U google-generativeai

print("✅ Dependencies installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 wh

In [3]:
import os
import google.generativeai as genai
from IPython.display import display, Markdown
from kaggle_secrets import UserSecretsClient
import pandas as pd
import json
from kaggle_secrets import UserSecretsClient
from google.generativeai.types import FunctionDeclaration, Tool


**Step2**: Setup API key

Get API KEY from Google AI Studio and add it in secrets

In [4]:
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("GOOGLE_API_KEY") 

genai.configure(api_key=my_secret)

print("✅ API Key securely loaded from Kaggle Secrets.")

✅ API Key securely loaded from Kaggle Secrets.


**Step3**: Define Knowledge Base & Tools

This creates the "Brain" of your agent. It defines a mock database of emergency supplies and the function (tool) the AI uses to access it.

In [5]:
supply_db = {
    "water": {"qty": 5000, "location": "Zone A (North)"},
    "medkits": {"qty": 150, "location": "Zone B (Hospital)"},
    "blankets": {"qty": 1000, "location": "Zone A (North)"},
    "generators": {"qty": 20, "location": "Zone C (Industrial)"}
}


def check_supplies(item_name: str):
    """
    Checks the inventory database for a specific item.
    Args:
        item_name: The item to search for (e.g., 'water', 'generators').
    """
    item = item_name.lower()
  
    for key in supply_db:
        if item in key or key in item:
            data = supply_db[key]
            return f"✅ IN STOCK: {data['qty']} units of {key} located at {data['location']}."
    
    return f"❌ OUT OF STOCK: We do not have '{item_name}' recorded in the database."


tools_list = [check_supplies]

print("✅ Tool 'check_supplies' registered.")

✅ Tool 'check_supplies' registered.


**Step4**: Initialize the Agent Context:

Configure the Gemini model to use the tools we just created.

In [6]:
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=tools_list,
    system_instruction="You are a Crisis Logistics AI. Your goal is to help rescue teams find supplies quickly. Always use the check_supplies tool before answering."
)


agent = model.start_chat(enable_automatic_function_calling=True)

print("🤖 Disaster Agent is Online.")

🤖 Disaster Agent is Online.


**Step5**: Capstone Demonstration

This loops through simulated user queries to prove the agent works autonomously.

In [7]:
requests = [
    "We have a power outage in the shelter. Do we have any generators?",
    "We need medical kits for the injured.",
    "Are there any inflatable boats in stock?"
]

display(Markdown("### 📡 Crisis Communication Log"))

for req in requests:
    display(Markdown(f"**👮 Rescue Team:** {req}"))
    

    response = agent.send_message(req)
    
    display(Markdown(f"**🤖 AI Agent:** {response.text}"))
    display(Markdown("---"))

### 📡 Crisis Communication Log

**👮 Rescue Team:** We have a power outage in the shelter. Do we have any generators?

**🤖 AI Agent:** Yes, we have 20 generators in stock at Zone C (Industrial).

---

**👮 Rescue Team:** We need medical kits for the injured.

**🤖 AI Agent:** Medical kits are out of stock.

---

**👮 Rescue Team:** Are there any inflatable boats in stock?

**🤖 AI Agent:** Inflatable boats are out of stock.

---

## 🏁 Conclusion 

### **Project Summary**
This Capstone demonstrates a working **AI Agent** that successfully:
1.  Identifying user intent ("Need water").
2.  Mapping intent to a technical tool (`check_inventory`).
3.  Retrieving structured data and converting it back to natural language.

### **Reflection**
This project applies the core concepts of the **Google x Kaggle AI Intensive**: Prompt Engineering, Function Calling, and Agentic Workflows. It moves beyond "Generative AI as a Creator" to "Generative AI as an Operator."


### 💖 Thank You, Google & Kaggle!
I am deeply grateful for the opportunity to participate in the **5-Day AI Intensive**. This course provided the essential bridge between AI theory and real-world application, making complex topics immediately actionable. The structure—delivered via Kaggle Notebooks—was perfect for hands-on learning.

### 🧠 Key Learnings & Value
This intensive transformed my approach to building with Large Language Models (LLMs). The most valuable skills gained include:

* **Autonomous Agents & Function Calling (Tool Use):** Mastering the architecture to enable LLMs (like the **Crisis Logistics Agent** in this project) to move beyond chatting and execute external Python functions.
* **Gemini SDK Mastery:** Gaining proficiency with the modern `google-generativeai` SDK for efficient model configuration and deployment.
* **Advanced Prompt Engineering:** Techniques for structuring prompts, defining specific roles (`system_instruction`), and managing conversation history.
* **RAG (Retrieval-Augmented Generation):** Understanding how to ground models in external data for factual accuracy.
* **Model Fine-Tuning:** Learning the principles behind customizing model behavior for specific tasks.

### 🚀 Final Assessment
The emphasis on practical skills, like designing the **Agentic workflow** demonstrated here, is what makes this course invaluable. It has directly equipped me with the tools to build, debug, and deploy sophisticated AI applications.

Thank you for providing such a high-quality, free, and deeply relevant educational experience!